# Model Prediction

We use Google Drive to share the data used for training the model and the model already trained. To add a shortcut to the location of the data in your Google Drive, follow these steps:    

* Go to https://drive.google.com/drive/folders/1EPfe4G6Y32c6eTrM6gnTjxr7KUlrGwAt?usp=sharing
* Click in "MAPBIOMAS-PUBLIC" &#8594; "Add Shortcut to Drive" &#8594; "My Drive" &#8594; "ADD SHORTCUT"

# Settings

#### Install libraries and mount drive

In [ ]:
!pip install retry

import logging
import json
from retry import retry
import ee
import multiprocessing
import os
import logging
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
from google.colab import drive, runtime
import shutil

In [ ]:
drive.mount("/content/drive")


# <span style="color:red">Google Earth Engine REST API functions</span>

This is an alternative way to download small images from Google Earth Engine without creating *tasks*, which requires a more advanced knowledge of the platform to implement. 

If you choose to download the images in the traditional way to a folder on Google Drive, skip the grouped cells here and change `PREDICT_INPUT_DIR` in settings to your folder directory.

#### Altenticar com o GEE

In [ ]:
ee.Authenticate()

ee.Initialize()

In [ ]:
KEY = "/content/drive/MyDrive/your_key.json"
EMAIL = "your_email@google.com"

PIXEL_SIZE = 30
CHIP_SIZE = 1024
BASE_PROJ = ee.Projection("EPSG:4326")

EXPORT_TO_TIFF = True

ASSETS_PATH = "users/your_user/path"

#### Conect with GCLoud json key

Attention! You will need a Google Cloud *service account* to follow this.

In [ ]:
!gcloud config set account {EMAIL}
!gcloud auth activate-service-account --key-file '{KEY}'

credentials = service_account.Credentials.from_service_account_file(KEY)
scoped_credentials = credentials.with_scopes(
    ["https://www.googleapis.com/auth/cloud-platform"]
)

session = AuthorizedSession(scoped_credentials)

In [ ]:
if True:  # Testing API connection
    response = session.get(
        "https://earthengine.googleapis.com/v1/projects/earthengine-public/assets/LANDSAT"
    )
    print(json.loads(response.content))

In [ ]:
url = "https://earthengine-highvolume.googleapis.com/v1/projects/earthengine-public/image:computePixels"
os.makedirs("results/", exist_ok=True)
os.makedirs("classifications/", exist_ok=True)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
fh = logging.FileHandler("app.log")
fh.setLevel(logging.INFO)
logger.addHandler(fh)

In [ ]:
# Landsat Collection 2
def mask_clouds(image):
    dilatedCloud = (1 << 1)
    cloud = (1 << 3)
    cloudShadow = (1 << 4)
    snow = (1 << 5)
    qa = image.select('BQA')
    mask = qa.bitwiseAnd(dilatedCloud)\
    .Or(qa.bitwiseAnd(cloud))\
    .Or(qa.bitwiseAnd(cloudShadow))\
    .Or(qa.bitwiseAnd(snow))
    
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(mask.Not()).updateMask(mask2)

In [ ]:
properties_dict = ee.Dictionary({
  'LANDSAT_4': {
    'C1': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'TIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0]
      },
    },
    'C2': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'TIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0]
      },
      'SR': {
        'bands': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1],
        'offset': [-0.2, -0.2, -0.2, -0.2, -0.2 ,-0.2 , 0]
      }
    }
  },
  'LANDSAT_5': {
    'C1': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'TIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0]
      },
    },
    'C2': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'TIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0]
      },
      'SR': {
        'bands': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1],
        'offset': [-0.2, -0.2, -0.2, -0.2, -0.2 ,-0.2 , 0]
      }
    }
  },
  'LANDSAT_7': {
    'C1': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'BQA'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0]
      },
    },
    'C2': {
      'TOA': {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0]
      },
      'SR': {
        'bands': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1],
        'offset': [-0.2, -0.2, -0.2, -0.2, -0.2 ,-0.2 , 0]
      }
    }
  },  
  'LANDSAT_8': {
    'C1': {
      'TOA': {
        'bands': ['B2', 'B3','B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'BQA'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TIR1', 'TIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0 ,0]
      },
    },
    'C2': {
      'TOA': {
        'bands': ['B2', 'B3','B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TIR1', 'TIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0 ,0]
      },
      'SR': {
        'bands': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1],
        'offset': [-0.2, -0.2, -0.2, -0.2, -0.2 ,-0.2 , 0]
      }
    }
  },  
  'LANDSAT_9': {
    'C2': {
      'TOA': {
        'bands': ['B2', 'B3','B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TIR1', 'TIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0 ,0]
      },
      'SR': {
        'bands': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1],
        'offset': [-0.2, -0.2, -0.2, -0.2, -0.2 ,-0.2 , 0]
      }
    }
  },  
  'Sentinel-2A': {
    'C1': {
      'TOA-SR': {
        'bands': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'QA60'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'RE1', 'RE2', 'RE3', 'NIR', 'RE4', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      }
    }
  },  
  'Sentinel-2B': {
    'C1': {
      'TOA-SR': {
        'bands': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'QA60'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'RE1', 'RE2', 'RE3', 'NIR', 'RE4', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      }
    }
  },
  'MODIS': {
    'C1': {
      'SR': {
        'bands': ['sur_refl_b03', 'sur_refl_b04', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b06', 'sur_refl_b07', 'StateQA'],
        'newBandNames': ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'BQA'],
        'scaleFactors': [1, 1, 1, 1, 1, 1, 1],
        'offset': [0, 0, 0, 0, 0, 0, 0]
      }
    }
  }
});

def get_landsat_properties(landsatImage):
    landsatProperties =  landsatImage.toDictionary(landsatImage.propertyNames())
  
    return landsatProperties\
        .set('SATELLITE_NAME', landsatProperties.getString('SPACECRAFT_ID'))\
        .set('CLOUD_COVER', landsatProperties.getNumber('CLOUD_COVER'))\
        .set('COLLECTION', 'C2')\
        .set('REFLECTANCE', ee.Algorithms.If(
                                            landsatImage.bandNames().containsAll(['SR_B2', 'SR_B3', 'SR_B4']),
                                            'SR',
                                            'TOA'
                                            ))


def get_sentinel_properties(sentinelImage):
    sentinelProperties = sentinelImage.toDictionary(sentinelImage.propertyNames())
  
    return sentinelProperties\
        .set('SATELLITE_NAME', sentinelProperties.getString('SPACECRAFT_NAME'))\
        .set('CLOUD_COVER', sentinelProperties.getNumber('CLOUDY_PIXEL_PERCENTAGE'))\
        .set('COLLECTION', 'C1')\
        .set('REFLECTANCE', 'TOA-SR')


def get_MODIS_properties(modisImage):
    return ee.Dictionary({
        'SATELLITE_NAME': 'MODIS',
        'COLLECTION': 'C1',
        'REFLECTANCE': 'SR'
    })


def get_properties(image):
    return ee.Dictionary(ee.Algorithms.If(ee.List(['LANDSAT_5', 'LANDSAT_7', 'LANDSAT_8', 'LANDSAT_9']).containsAll([image.get('SPACECRAFT_ID')]), 
                        get_landsat_properties(image), 
                        ee.Algorithms.If(ee.List(['Sentinel-2A', 'Sentinel-2B']).containsAll([image.get('SPACECRAFT_NAME')]),
                            get_sentinel_properties(image),
                            get_MODIS_properties(image)
                            )
                        ))


def standardize_image(image):
    imageProperties = get_properties(image)

    properties = ee.Dictionary(ee.Dictionary(ee.Dictionary(
                        properties_dict\
                        .get(imageProperties.get('SATELLITE_NAME')))\
                        .get(imageProperties.get('COLLECTION')))\
                        .get(imageProperties.get('REFLECTANCE')))\

    renamedImage = image.select(properties.get('bands'), properties.get('newBandNames'))

    reescaledImage = renamedImage.multiply(ee.Number(ee.List(properties.get('scaleFactors')))).add(ee.Number(ee.List(properties.get('offset'))))
    
    return ee.Image(reescaledImage.setMulti(imageProperties))


#### Install GDAL

In [ ]:
!sudo apt-get install gdal-bin -y
!sudo add-apt-repository ppa:ubuntugis/ppa -y
!sudo apt-get update -y
!pip install retry rasterio
import rasterio as rio

In [ ]:
def prepareImages(assets_path):
    """Generates a list of files to be downloaded."""
    points = (
        ee.FeatureCollection(assets_path)
        .map(lambda ft: ft.geometry().centroid())
        .aggregate_array(".geo")
        .getInfo()
    )
    squares = [
        ee.Geometry.Point(ft["coordinates"]).buffer(PIXEL_SIZE * CHIP_SIZE / 2).bounds()
        for ft in points
    ]

    return list(enumerate(squares))


@retry(tries=10, delay=5, backoff=3)
def downloadEachImage (args):

  """Download each file of list."""
  
  start = str(years[x])+'-01-01' 
  end = str(years[x])+'-12-31' 
  cloud_cover = 80
  
  index = args[0]
  region = args[1]
  
  filters = ee.Filter.And(\
      ee.Filter.bounds(region),\
      ee.Filter.date(start, end),\
      ee.Filter.lt('CLOUD_COVER', cloud_cover)
  )
  
  ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_TOA").filter(filters)
  ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filter(filters)
  ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filter(filters)
  ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA").filter(filters)
  
  expression = (
      ls5.merge(ls7).merge(ls8).merge(ls9)\
      .map(standardize_image)\
      .map(mask_clouds)\
      .select(["RED", "NIR", "SWIR1"])\ 
      .median()\
      .reproject(BASE_PROJ.crs(), None, PIXEL_SIZE)
  )
  # Get scales out of the transform.
  proj = ee.Projection("EPSG:4326").atScale(PIXEL_SIZE).getInfo()
  scale_y = -proj["transform"][0]
  scale_x = proj["transform"][4]

  # Extraction parameters
  listCoords = ee.Array.cat(region.coordinates(), 1).getInfo()

  # coords
  xMin = listCoords[0][0]
  yMax = listCoords[2][1]
  coords = [xMin, yMax]

  grid = {
        "dimensions": {"width": CHIP_SIZE, "height": CHIP_SIZE},
        "affineTransform": {
            "scaleX": scale_x,
            "scaleY": scale_y,
            "translateX": coords[0],
            "translateY": coords[1],
        },
        "crsCode": "EPSG:4326",
    }

  response = session.post(
        url=url,
        data=json.dumps(
            {
                "expression": ee.serializer.encode(expression),
                "fileFormat": "GEO_TIFF" if EXPORT_TO_TIFF else "NPY",
                "bandIds": expression.bandNames().getInfo(),
                "grid": grid,
            }
        ),
    )

  try:
        error = json.loads(response.content)
        logger = logging.getLogger()
        logger.error(str(json.loads(response.content)))
        raise Exception("run again")
  except UnicodeDecodeError:
        if EXPORT_TO_TIFF:
            with rio.io.MemoryFile(response.content) as memfile:
                with memfile.open() as input:
                    with rio.open(
                        f"results/mosaic_{index}_{str(years[x])}.tiff",
                        "w",
                        **dict(
                            input.meta,
                            compress="DEFLATE",
                            nodata=-1,
                        ),
                    ) as output:
                        output.write(input.read())
        else:
            with open(f"results/mosaic_{index}.npy", "wb") as file:
                file.write(response.content)

## Requests

In [ ]:
years = list(range(1985,2023)) # ------------> choose the range of years to download images

In [ ]:
for x in range(0, len(years)):
  items = prepareImages(ASSETS_PATH)

  with multiprocessing.Pool() as pool:
      count = 0
      for _ in pool.imap_unordered(
          downloadEachImage, items, chunksize=max(len(items) // os.cpu_count(), 1)
      ):
          pass

In [ ]:
!du -h results

1.1G	results


# Settings for Model and Predictions

### Install GDAL and environment Setting 

In [ ]:
#Step 1
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y

import random
import numpy as np
seed = 426
random.seed(seed)
np.random.seed = seed

## Settings

In [ ]:
CHIP_SIZE = 256
CHANNELS = 3
LABELS = [0, 1]
SPATIAL_SCALE = 30
PROJECTION = 3857
REPROJECT = False


# Build Datasets
GRIDS = 2
ROTATE = True
FLIP = True

PUBLIC_ROOT_DIR ="/content/drive/My Drive/MAPBIOMAS-PUBLIC/collection_8/oil_palm" 
DATASET_DIR = None

# Train model
# in case of error during training, decrease the value of TRAIN_BATCH_SIZE
TRAIN_BATCH_SIZE = 8
TRAIN_EPOCHS = 100

# Predict images
PREDICT_INPUT_DIR = "/content/results" # <-- CHANGE THIS TO YOUR DRIVE SAMPLES FOLDER PATH IF YOU SKIPPED THE "Google Earth Engine REST API functions" BLOCK. 
PREDICT_CHIP_SIZE = 256
PREDICT_GRIDS = 3
PREDICT_BATCH_SIZE = 1
PREDICT_OUPUT_DIR = "/content/classifications"

# Load trained model
MODEL_DIR = f"{PUBLIC_ROOT_DIR}/logs"

## Image Utils

In [ ]:
# -*- encoding=UTF-8-*-

import gc
import os

import h5py
import numpy as np
import psutil
from osgeo import gdal, osr
from skimage.transform import resize, rotate
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def load_file(path, resizeTo=None, norm=False):
  
    original_source = gdal.Open(path)

    if REPROJECT:
        new_source = reproject_dataset(original_source,
                                    pixel_spacing=SPATIAL_SCALE,
                                    epsg_to=PROJECTION)
    else:
        dataSource = original_source

    if not dataSource is None:
        bands = []
        for index in range(1, dataSource.RasterCount + 1):
            band = dataSource.GetRasterBand(index).ReadAsArray()
            if norm:
                normalized_band = normalize(band)
                bands.append(normalized_band)
            else:
                bands.append(band)

        image = np.dstack(bands)

        return image
    else:
        return None


def reproject_dataset ( dataset, pixel_spacing=30., epsg_to=3857 ):
    """
    A sample function to reproject and resample a GDAL dataset from within 
    Python. The idea here is to reproject from one system to another, as well
    as to change the pixel size. The procedure is slightly long-winded, but
    goes like this:
    
    1. Set up the two Spatial Reference systems.
    2. Open the original dataset, and get the geotransform
    3. Calculate bounds of new geotransform by projecting the UL corners 
    4. Calculate the number of pixels with the new projection & spacing
    5. Create an in-memory raster dataset
    6. Perform the projection
    """
    # We now open the dataset
    g = gdal.Open ( dataset )

    # Define the UK OSNG, see <http://spatialreference.org/ref/epsg/27700/>
    osng = osr.SpatialReference ()
    osng.ImportFromEPSG ( epsg_to )

    wkt = g.GetProjection()
    wgs84 = osr.SpatialReference ()
    wgs84.ImportFromWkt(wkt)

    tx = osr.CoordinateTransformation ( wgs84, osng )
    # Up to here, all  the projection have been defined, as well as a 
    # transformation from the from to the  to :)

    # Get the Geotransform vector
    geo_t = g.GetGeoTransform ()
    x_size = g.RasterXSize # Raster xsize
    y_size = g.RasterYSize # Raster ysize
    # Work out the boundaries of the new dataset in the target projection
    (ulx, uly, ulz ) = tx.TransformPoint( geo_t[0], geo_t[3])
    (lrx, lry, lrz ) = tx.TransformPoint( geo_t[0] + geo_t[1]*x_size, \
                                          geo_t[3] + geo_t[5]*y_size )
    # See how using 27700 and WGS84 introduces a z-value!
    # Now, we create an in-memory raster
    mem_drv = gdal.GetDriverByName( 'MEM' )
    # The size of the raster is given the new projection and pixel spacing
    # Using the values we calculated above. Also, setting it to store one band
    # and to use Float32 data type.
    dest = mem_drv.Create('', int((lrx - ulx)/pixel_spacing), \
            int((uly - lry)/pixel_spacing), g.RasterCount, gdal.GDT_Float32)
    # Calculate the new geotransform
    new_geo = ( ulx, pixel_spacing, geo_t[2], \
                uly, geo_t[4], -pixel_spacing )
    # Set the geotransform
    dest.SetGeoTransform( new_geo )
    dest.SetProjection ( osng.ExportToWkt() )
    # Perform the projection/resampling 
    res = gdal.ReprojectImage( g, dest, \
                wgs84.ExportToWkt(), osng.ExportToWkt(), \
                gdal.GRA_Bilinear )
    return dest


def normalize(image):
    image_max = float(np.max(image))
    image_min = float(np.min(image))
    normalized = (image - image_min) / (image_max - image_min)

    del image
    gc.collect()
    return normalized


def get_rotate(image):
    images = []
    for rot in [90, 180, 270]:
        image_rotate = rotate(image, rot, preserve_range=True)
        images.append(image_rotate)

    del image
    gc.collect()
    return images


def get_flip(image):
    horizontal_flip = image[:, ::-1]
    vertical_flip = image[::-1, :]

    del image
    gc.collect()
    return [horizontal_flip, vertical_flip]


def load_dataset(dataset, read_only=False):
    if read_only:
        dataset = h5py.File(dataset, 'r')
    else:
        dataset = h5py.File(dataset, 'r+')
    x_data = dataset["x"]
    y_data = dataset["y"]
    return dataset, x_data, y_data


def make_dataset(filename, width, height, channels):
    dataset = h5py.File(filename, 'w')
    x_data = dataset.create_dataset("x", (0, width, height, channels), 'f',
                                    maxshape=(None, width, height, channels),
                                    chunks=True
                                    # , compress="gzip"
                                    )
    y_data = dataset.create_dataset("y", (0, width, height, 1), 'f',
                                    maxshape=(None, width, height, 1),
                                    chunks=True
                                    # , compress="gzip"
                                    )
    return dataset, x_data, y_data


def get_available_memory():
    return 100 - psutil.virtual_memory().percent


def chip_is_empty(chip):
    labels_unique = np.unique(chip)
    if 0 in labels_unique and len(labels_unique) == 1:
        return True
    else:
        return False


def generate_dataset(image_path, labels_path, train_path, validation_path, 
                     chip_size, channels, grids=1, rotate=False, flip=False):
    image_data = load_file(image_path, norm=True)
    image_labels = load_file(labels_path)

    # resize labels
    image_labels = resize(image_labels,
                          (image_data.shape[0], image_data.shape[1]),
                          preserve_range=True, anti_aliasing=True).astype(np.int8)

    image = np.dstack([image_data, image_labels])

    # pad image to avoid the loss of border samples
    del image_data
    del image_labels
    gc.collect()

    X_set = []
    y_set = []
    for step in get_grids(grids, chip_size):
        for (x, y, window, dimension) in sliding_window(image,
                                                        step["steps"],
                                                        step["chip_size"],
                                                        (
                                                                chip_size,
                                                                chip_size)):
            
            train = np.array(window[:, :, : channels], dtype=np.float16)
            labels = np.array(window[:, :, -1:], dtype=np.int8)

            if not chip_is_empty(labels):
                raw_image = np.dstack([train, labels])
                images_daugmentation = [raw_image]

                if rotate:
                    images_rotate = get_rotate(raw_image)
                    images_daugmentation.extend(images_rotate)

                if flip:
                    images_flip = []
                    for im in images_daugmentation:
                        images_flip.extend(get_flip(im))
                    images_daugmentation.extend(images_flip)

                for i in images_daugmentation:
                    new_train = np.array(i[:, :, :channels], dtype=np.float16)
                    new_labels = np.array(i[:, :, -1:], dtype=np.int8)

                    np.clip(new_labels, 0, None, out=new_labels)
                    X_set.append(new_train)
                    y_set.append(new_labels)
        
        print("\nAll:", len(X_set))

        if len(X_set) >= 5:
            X_train, X_val, y_train, y_val = train_test_split(X_set, y_set, test_size=0.25, random_state=1)
            
            print("X_train:", len(X_train))
            print("X_val:", len(X_val))

            save_dataset(X_train, y_train, train_path, chip_size, channels)
            save_dataset(X_val, y_val, validation_path, chip_size, channels)


def save_dataset(X, y, output_path, chip_size, channels):
    if os.path.isfile(output_path):
        dataset, x_data, y_data = load_dataset(output_path)
    else:
        dataset, x_data, y_data = make_dataset(output_path, chip_size,
                                               chip_size, channels)

    length = len(X)

    x_data_size = x_data.len()
    y_data_size = y_data.len()

    x_data.resize((x_data_size + length, chip_size, chip_size, channels))
    y_data.resize((y_data_size + length, chip_size, chip_size, 1))

    print("Saving dataset...")
    
    x_data[x_data_size:] = X
    y_data[y_data_size:] = y
    
    dataset.close()
    del x_data
    del y_data
    gc.collect()


def sliding_window(image, step, chip_size, chip_resize):
    # slide a chip across the image
    step_cols = int(step[0])
    step_rows = int(step[1])

    cols = image.shape[1]
    rows = image.shape[0]

    chip_size_cols = chip_size[0]
    chip_size_rows = chip_size[1]

    chip_resize_cols = chip_resize[0]
    chip_resize_rows = chip_resize[1]

    for y in range(0, rows, step_rows):
        for x in range(0, cols, step_cols):

            origin_x = x
            origin_y = y

            if (origin_y + chip_size_rows) > rows:
                origin_y = rows - chip_size_rows

            if (origin_x + chip_size_cols) > cols:
                origin_x = cols - chip_size_cols

            chip = image[origin_y:origin_y + chip_size_rows,
                   origin_x: origin_x + chip_size_cols]

            original_shape = chip.shape

            if chip.shape != (chip_resize_cols, chip_resize_rows):
                chip = resize(chip,
                              (chip_resize_cols, chip_resize_rows),
                              preserve_range=True,
                              anti_aliasing=True).astype(np.float16)

            yield (origin_x, origin_y, chip, original_shape)


def get_window(matrix, x, y, width, height):
    return matrix[y:y + height, x:x + width]


def set_window(matrix, x, y, new_matrix):
    for i_index, i in enumerate(range(y, y + new_matrix.shape[0])):
        for j_index, j in enumerate(range(x, x + new_matrix.shape[1])):
            matrix[i][j] = new_matrix[i_index][j_index]


def get_grids(grids, chip_size):
    # multiplying chip_size by 1.5 to prevent missing data in rotation
    grids_dict = {
        1: [
            {"steps": (chip_size, chip_size),
             "chip_size": (chip_size, chip_size)}
        ],
        2: [
            {"steps": (int(chip_size * 0.5), int(chip_size * 0.5)),
             "chip_size": (chip_size, chip_size)},
        ],
        3: [
            {"steps": (int(chip_size * 0.9), int(chip_size * 0.9)),
             "chip_size": (chip_size, chip_size)},
        ]
    }

    return grids_dict[grids]

### U-Net

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation


def conv(n_filters, kernel_size, activation='elu', inputs=None):
    net = Conv2D(n_filters, kernel_size, activation=None, kernel_initializer='he_normal', padding='same') (inputs) 
    net = BatchNormalization()(net)
    net = Activation(activation)(net)
    return net

def transpose(n_filters, kernel_size, activation='elu', inputs=None):
    net = Conv2DTranspose(n_filters, kernel_size, activation=None, strides=(2, 2), padding='same', kernel_initializer='he_normal') (inputs)
    net = BatchNormalization()(net)
    net = Activation(activation)(net)
    return net

def model_fn(input_shape, n_filters=64):
    inputs = keras.Input(input_shape)

    c1 = conv(n_filters * 1, (3, 3), inputs=inputs)
    c1 = conv(n_filters * 1, (3, 3), inputs=c1)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(0.25) (p1)

    c2 = conv(n_filters * 2, (3, 3), inputs=p1)
    c2 = conv(n_filters * 2, (3, 3), inputs=c2)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(0.25) (p2)

    c3 = conv(n_filters * 4, (3, 3), inputs=p2)
    c3 = conv(n_filters * 4, (3, 3), inputs=c3)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(0.5) (p3)

    c4 = conv(n_filters * 8, (3, 3), inputs=p3)
    c4 = conv(n_filters * 8, (3, 3), inputs=c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(0.5) (p4)

    c5 = conv(n_filters * 16, (3, 3), inputs=p4)
    c5 = conv(n_filters * 16, (3, 3), inputs=c5)
    c5 = Dropout(0.5) (c5)

    u6 = transpose(n_filters * 8, (2, 2), inputs=c5)
    u6 = concatenate([u6, c4])
    c6 = conv(n_filters * 8, (2, 2), inputs=u6)
    c6 = conv(n_filters * 8, (2, 2), inputs=c6)
    c6 = Dropout(0.5) (c6)
    
    u7 = transpose(n_filters * 4, (2, 2), inputs=c6)
    u7 = concatenate([u7, c3])
    c7 = conv(n_filters * 4, (2, 2), inputs=u7)
    c7 = conv(n_filters * 4, (2, 2), inputs=c7)
    c7 = Dropout(0.5) (c7)

    u8 = transpose(n_filters * 2, (2, 2), inputs=c7)
    u8 = concatenate([u8, c2])
    c8 = conv(n_filters * 2, (2, 2), inputs=u8)
    c8 = conv(n_filters * 2, (2, 2), inputs=c8)
    c8 = Dropout(0.25) (c8)
    
    u9 = transpose(n_filters * 1, (2, 2), inputs=c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv(n_filters * 1, (2, 2), inputs=u9)
    c9 = conv(n_filters * 1, (2, 2), inputs=c9)
    c9 = Dropout(0.25) (c9)

    outputs = conv(1, (1, 1), activation='sigmoid', inputs=c9)

    model = keras.Model(inputs=inputs, outputs=outputs)

    # começa o treinamento com 0.01, depois de perceber que estáá começando a ficar instáável, 
    # altera para 0.0001
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.0001) # 0.01 or 0.0001

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[iou])

    return model

## Classifier

In [ ]:
import numpy as np
import tensorflow as tf
from osgeo import gdal
from tqdm import tqdm

class Classifier(object):
    def __init__(self, model, model_dir):
        self.__model = model
        self.__model.summary()

        checkpoint_path = "{dir}/model.ckpt".format(dir=model_dir)

        latest = tf.train.latest_checkpoint(model_dir)

        if latest:
            model.load_weights(latest)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=True,
            save_best_only=True)

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=model_dir)

        self.__callbacks = [cp_callback, tensorboard_callback]

    def train(self, input_train, input_validation, epochs, batch_size):
        train_file, train_data, train_labels = load_dataset(input_train, read_only=True)
        validation_file, validation_data, validation_labels = load_dataset(input_validation, read_only=True)

        train_images = np.asarray(train_data[:8000], dtype=np.float16)
        train_labels = np.asarray(train_labels[:8000], dtype=np.int8)
        
        validation_images = np.asarray(validation_data[:2000], dtype=np.float16)
        validation_labels = np.asarray(validation_labels[:2000], dtype=np.int8)
        
        print("\nTrain: {0}\n".format(train_images.shape[0]))
        print("Validation: {0}\n".format(validation_images.shape[0]))

        return self.__model.fit(x=train_images, 
                         y=train_labels,
                         validation_data=(validation_images, validation_labels),
                         epochs=epochs,
                         batch_size=batch_size,
                         verbose=1,
                         callbacks=self.__callbacks,
                         shuffle=True)
      

    

    def evaluate(self, input_test, batch_size):
        test_file, test_data, test_labels = load_dataset(input_test, read_only=True)

        test_data = np.asarray(test_data, dtype=np.float16)
        test_labels = np.asarray(test_labels, dtype=np.int8)

        test_results = self.__model.evaluate(test_data,
                                             test_labels,
                                             batch_size=batch_size)
        
        print('test loss, test acc:', test_results)

    def predict(self, input_path, output_path, chip_size, channels, grids, batch_size):

        input_dataset = gdal.Open(input_path)

        image = load_file(input_path, norm=True)[:, :, :channels]

        predicted_image = np.zeros((image.shape[0], image.shape[1]), dtype=np.int8)

        grids = get_grids(grids, chip_size)

        for step in grids:
            batch = []
            windows = sliding_window(image, step["steps"], step["chip_size"],
                (chip_size, chip_size))

            for (x, y, chip, original_dimensions) in tqdm(iterable=windows,
                                                          miniters=10,
                                                          unit=" windows"):

                normalized_chip = normalize(chip)

         
                batch.append({
                    "chip": normalized_chip,
                    "x": x,
                    "y": y,
                    "dimensions": original_dimensions
                })

                if len(batch) >= batch_size:
                    chips = []
                    positions = []
                    dimensions = []

                    for b in batch:
                        chips.append(b.get("chip"))
                        positions.append((b.get("x"), b.get("y")))
                        dimensions.append(b.get("dimensions"))

                    chips = np.array(chips, dtype=np.float16)

                    pred = self.__model.predict(chips)

                    for chip, position, dimension, predict in zip(chips,
                                                                  positions,
                                                                  dimensions,
                                                                  pred):
                        predict[predict > 0.5] = 1
                        predict[predict <= 0.5] = 0

                        predict = resize(predict, (dimension[0], dimension[1]),
                            preserve_range=True,
                            anti_aliasing=True).astype(np.int8)

                        predict = predict.reshape(
                            (predict.shape[0], predict.shape[1]))

                        predicted = get_window(predicted_image,
                                               position[0],
                                               position[1],
                                               predict.shape[1],
                                               predict.shape[0])

                        if predict.shape != predicted.shape:
                            raise Exception("predict.shape != predicted.shape")

                        set_window(predicted_image,
                                   position[0],
                                   position[1],
                                   np.add(predict, predicted))
                    batch = []
            print("Saving results...")
            driver = input_dataset.GetDriver()
            output_dataset = driver.Create(output_path,
                                           image.shape[1],
                                           image.shape[0],
                                           1,
                                           gdal.GDT_Int16,
                                           ['COMPRESS=DEFLATE'])
            output_dataset.SetGeoTransform(input_dataset.GetGeoTransform())
            output_dataset.SetProjection(input_dataset.GetProjection())
            output_dataset.GetRasterBand(1) \
                .WriteArray(predicted_image.reshape((predicted_image.shape[0],
                                                     predicted_image.shape[1])
                                                    ), 0, 0)
            output_dataset.FlushCache()
            output_dataset = None
            print("The results have been saved!")

## Metrics

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from os import listdir
from os.path import isfile, join, sep, exists

def iou(y_true, y_pred):
    y_pred = K.cast(K.greater(y_pred, .5), dtype='float32') # .5 is the threshold
    inter = K.sum(K.sum(K.squeeze(y_true * y_pred, axis=3), axis=2), axis=1)
    union = K.sum(K.sum(K.squeeze(y_true + y_pred, axis=3), axis=2), axis=1) - inter
    acc = K.mean((inter + K.epsilon()) / (union + K.epsilon()))
    return acc

## Prediction

In [ ]:
from os import listdir
from os.path import isfile, join, sep, exists

model = model_fn((PREDICT_CHIP_SIZE, PREDICT_CHIP_SIZE, CHANNELS))

classifier = Classifier(model=model, model_dir=MODEL_DIR)

if not os.path.exists(PREDICT_INPUT_DIR):
    print("Please wait until Google Earth Engine finishes processing your task.")

files = [f for f in listdir(PREDICT_INPUT_DIR) if isfile(join(PREDICT_INPUT_DIR, f))]

if len(files) == 0:
    print("No file found.")

for f in files:
    print("File:", f)
    input_file = "{directory}{sep}{filepath}".format(directory=PREDICT_INPUT_DIR,
                                                     sep=sep,
                                                     filepath=f)

    output_file = "{directory}{sep}{filepath}".format(directory=PREDICT_OUPUT_DIR,
                                                      sep=sep,
                                                      filepath=f)

    if not os.path.exists(PREDICT_OUPUT_DIR):
            os.makedirs(PREDICT_OUPUT_DIR)

    if exists(output_file):
        print("File {} exists.".format(output_file))
        continue
        
    print("Predict: ", input_file, "  >>  ", output_file)

    classifier.predict(
        input_path=input_file,
        output_path=output_file,
        chip_size=PREDICT_CHIP_SIZE,
        channels=CHANNELS,
        grids=PREDICT_GRIDS,
        batch_size=PREDICT_BATCH_SIZE
    )

# Create Merge

In [ ]:
!cd classifications && gdalbuildvrt tmp.vrt *_2022.tiff -hidenodata -srcnodata 0
!cd classifications && GDAL_CACHEMAX=512 gdal_translate -of GTiff -a_nodata 0 -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES  tmp.vrt merged_2022_2_tiles.tiff

## Export merge

In [ ]:
from google.colab import files
files.download('/content/classifications/merged_2022_2_tiles.tiff')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

If you want to delete your results from colabs temporary storage, uncomment and run the cell bellow.

In [ ]:
#os.system("rm -r /content/classifications")
#os.system("rm -r /content/results")
#os.system("mkdir /content/classifications")
#os.system("mkdir /content/results")